In [1]:
import numpy as np
nnos = 4
coord = np.array([[0.0,0.0],[10.0,0.0],[0.0,10.0],[10.0,10.0]]) #cx,cy
nel = 2
conec = np.array([[0,1,3],[0,3,2]]) #no1,no2,no3
mat = np.array([[1000.0,0.25],[2000.0,0.30]]) #E,ni
esp = np.array([0.1,0.2]) #esp
prop = np.array([[0,0],[1,1]]) #mat,esp
vinc = np.array([[0,0,0.0],[0,1,0.0],[1,1,0.0]]) #no,dir (0->x e 1->y),val
forca = np.array([[2,1,1.0],[3,1,1.0]]) #no,dir,val
dist = np.array([[0,10.0,10.0],[1,10.0,10.0]])

In [2]:
dist_elem = np.zeros((nel,2))
for i in range(dist.shape[0]):
    el = int(dist[i,0])
    qx = dist[i,1]
    qy = dist[i,2]
    dist_elem[el,0] = qx
    dist_elem[el,1] = qy

npe = 3

ksieta = np.array([[1.0,0.0],
                   [0.0,1.0],
                   [0.0,0.0]
                  ])

nph = 12
hammer = {
            "ksi":[0.501426509658179, 0.249286745170910, 0.249286745170910,
                    0.873821971016996, 0.063089014491502, 0.063089014491502,
                    0.053145049844816, 0.310352451033785, 0.636502499121399,
                    0.310352451033785, 0.636502499121399, 0.053145049844816],
            "eta":[0.249286745170910, 0.249286745170910, 0.501426509658179,
                    0.063089014491502, 0.063089014491502, 0.873821971016996,
                    0.310352451033785, 0.636502499121399, 0.053145049844816,
                    0.053145049844816, 0.310352451033785, 0.636502499121399],
            "peso":[0.116786275726379/2.0, 0.116786275726379/2.0, 0.116786275726379/2.0,
                     0.050844906370207/2.0, 0.050844906370207/2.0, 0.050844906370207/2.0,
                     0.082851075618374/2.0, 0.082851075618374/2.0, 0.082851075618374/2.0,
                     0.082851075618374/2.0, 0.082851075618374/2.0, 0.082851075618374/2.0]
         }

ep = 0 #0 -> EPT, 1 -> EPD
kglobal = np.zeros((2*nnos,2*nnos))
fglobal = np.zeros(2*nnos)

def fforma(ksi,eta):
    fi = np.array([ksi,eta,1-ksi-eta])
    dfidksi = np.array([1,0,-1])
    dfideta = np.array([0,1,-1])
    return fi,dfidksi,dfideta

for iel in range(nel):
    young = mat[int(prop[iel,0]),0]
    ni = mat[int(prop[iel,0]),1]
    h = esp[int(prop[iel,1])]
    qx = dist_elem[iel,0]
    qy = dist_elem[iel,1]
    if ep == 0: #EPT
        D11 = D22 = young / (1 - ni**2)
        D12 = ni * young / (1 - ni**2)
    else:
        D11 = D22 = (1 - ni) * young / (1 + ni) / (1 - 2 * ni)
        D12 = ni * young / (1 + ni) / (1 - 2 * ni)
    D33 = young / 2 / (1 + ni)
    mxx = np.array([[D11,0],[0,D33]])
    myy = np.array([[D33,0],[0,D22]])
    mxy = np.array([[0,D12],[D33,0]])
    myx = np.array([[0,D33],[D12,0]])

    klocal = np.zeros((2*npe,2*npe))
    flocal = np.zeros(2*npe)
    for ih in range(nph):
        ksi = hammer["ksi"][ih]
        eta = hammer["eta"][ih]
        peso = hammer["peso"][ih]
        fi,dfidksi,dfideta = fforma(ksi,eta)
        dfi = np.zeros((4,2*npe))
        for i in range(npe):
            dfi[0,2*i] = dfidksi[i]
            dfi[1,2*i+1] = dfidksi[i]
            dfi[2,2*i] = dfideta[i]
            dfi[3,2*i+1] = dfideta[i]
        dxdksi = 0
        dxdeta = 0
        dydksi = 0
        dydeta = 0
        for i in range(npe):
            dxdksi += dfidksi[i] * coord[int(conec[iel,i]),0]
            dxdeta += dfideta[i] * coord[int(conec[iel,i]),0]
            dydksi += dfidksi[i] * coord[int(conec[iel,i]),1]
            dydeta += dfideta[i] * coord[int(conec[iel,i]),1]
        jac = np.array([[dxdksi,dxdeta],[dydksi,dydeta]])
        det = np.linalg.det(jac)
        jacinv = np.linalg.inv(jac)
        dksidx = jacinv[0,0]
        dksidy = jacinv[0,1]
        detadx = jacinv[1,0]
        detady = jacinv[1,1]
        dx = np.zeros((2,4))
        dx[0,0] = dksidx
        dx[0,2] = detadx
        dx[1,1] = dksidx
        dx[1,3] = detadx
        dy = np.zeros((2,4))
        dy[0,0] = dksidy
        dy[0,2] = detady
        dy[1,1] = dksidy
        dy[1,3] = detady
        flocal += det * peso * np.array([qx * fi[0],qy * fi[0],
                                         qx * fi[1],qy * fi[1],
                                         qx * fi[2],qy * fi[2]])
        klocal += h * det * peso * np.dot(dfi.transpose(),
                                      np.dot(
                                         np.dot(dx.transpose(),np.dot(mxx,dx)) +
                                         np.dot(dy.transpose(),np.dot(myy,dy)) +
                                         np.dot(dx.transpose(),np.dot(mxy,dy)) +
                                         np.dot(dy.transpose(),np.dot(myx,dx)),
                                         dfi
                                         )
                                      )
    for il in range(npe):
        for jl in range(2):
            fglobal[2 * int(conec[iel,il]) + jl] += flocal[2 * il + jl]
            for ic in range(npe):
                for jc in range(2):
                    kglobal[2*int(conec[iel,il])+jl,2*int(conec[iel,ic])+jc] += klocal[2*il+jl,2*ic+jc]
print(kglobal)
                    
for i in range(forca.shape[0]):
    no = int(forca[i,0])
    direc = int(forca[i,1])
    valor = forca[i,2]
    fglobal[2 * no + direc] += valor

for i in range(vinc.shape[0]):
    no = int(vinc[i,0])
    direc = int(vinc[i,1])
    valor = vinc[i,2]
    fglobal -= valor * kglobal[:,2 * no + direc]
    fglobal[2 * no + direc] = valor
    kglobal[2 * no + direc,:] = 0.0
    kglobal[:,2 * no + direc] = 0.0
    kglobal[2 * no + direc,2* no + direc] = 1.0
print(kglobal)

desl = np.linalg.solve(kglobal,fglobal)

tensao = np.zeros((nnos,4))#nelem,sigx,sigy,talxy

for iel in range(nel):
    print('Elemento')
    print(iel)
    young = mat[int(prop[iel,0]),0]
    ni = mat[int(prop[iel,0]),1]
    h = esp[int(prop[iel,1])]
    if ep == 0: #EPT
        D11 = D22 = young / (1 - ni**2)
        D12 = ni * young / (1 - ni**2)
    else:
        D11 = D22 = (1 - ni) * young / (1 + ni) / (1 - 2 * ni)
        D12 = ni * young / (1 + ni) / (1 - 2 * ni)
    D33 = young / 2 / (1 + ni)

    for ih in range(npe):
        print('ponto nodal')
        print(ih)
        ksi = ksieta[ih,0]
        eta = ksieta[ih,1]
        fi,dfidksi,dfideta = fforma(ksi,eta)
        dfi = np.zeros((4,2*npe))
        for i in range(npe):
            dfi[0,2*i] = dfidksi[i]
            dfi[1,2*i+1] = dfidksi[i]
            dfi[2,2*i] = dfideta[i]
            dfi[3,2*i+1] = dfideta[i]
        dxdksi = 0
        dxdeta = 0
        dydksi = 0
        dydeta = 0
        for i in range(npe):
            dxdksi += dfidksi[i] * coord[int(conec[iel,i]),0]
            dxdeta += dfideta[i] * coord[int(conec[iel,i]),0]
            dydksi += dfidksi[i] * coord[int(conec[iel,i]),1]
            dydeta += dfideta[i] * coord[int(conec[iel,i]),1]
        jac = np.array([[dxdksi,dxdeta],[dydksi,dydeta]])
        jacinv = np.linalg.inv(jac)
        dksidx = jacinv[0,0]
        dksidy = jacinv[0,1]
        detadx = jacinv[1,0]
        detady = jacinv[1,1]
        dx = np.zeros((2,4))
        dx[0,0] = dksidx
        dx[0,2] = detadx
        dx[1,1] = dksidx
        dx[1,3] = detadx
        dy = np.zeros((2,4))
        dy[0,0] = dksidy
        dy[0,2] = detady
        dy[1,1] = dksidy
        dy[1,3] = detady

        edesl = np.zeros(2*npe)
        for i in range(npe):
            edesl[2*i] = desl[2*int(conec[iel,i])]
            edesl[2*i+1] = desl[2*int(conec[iel,i])+1]

        vdksideta = np.dot(dfi,edesl)
        ddx = np.dot(dx,vdksideta) #dudx dvdx
        ddy = np.dot(dy,vdksideta) #dudy dvdy

        veps = np.zeros(3)
        md = np.array([[D11,D12,  0  ],
                       [D12,D22,  0  ],
                       [ 0 , 0 ,2*D33]
                      ])
        veps[0] = ddx[0]
        veps[1] = ddy[1]
        veps[2] = 0.5 * (ddy[0] + ddx[1])
        print ('vetor de deformações elementar')
        print (veps)
        vsig = np.dot(md,veps)
        print ('vetor de tensões elementar')
        print (vsig)
        tensao[int(conec[iel,ih]),0] += 1
        tensao[int(conec[iel,ih]),1] += vsig[0]
        tensao[int(conec[iel,ih]),2] += vsig[1]
        tensao[int(conec[iel,ih]),3] += vsig[2]
        print ('vetor de tensões acumulado')
        print (tensao)

tensao[:,1] /= tensao[:,0]
tensao[:,2] /= tensao[:,0]
tensao[:,3] /= tensao[:,0]

[[ 1.30256410e+02  0.00000000e+00 -5.33333333e+01  1.33333333e+01
  -7.69230769e+01  7.69230769e+01  0.00000000e+00 -9.02564103e+01]
 [ 0.00000000e+00  2.39780220e+02  2.00000000e+01 -2.00000000e+01
   6.59340659e+01 -2.19780220e+02 -8.59340659e+01  0.00000000e+00]
 [-5.33333333e+01  2.00000000e+01  7.33333333e+01 -3.33333333e+01
   0.00000000e+00  0.00000000e+00 -2.00000000e+01  1.33333333e+01]
 [ 1.33333333e+01 -2.00000000e+01 -3.33333333e+01  7.33333333e+01
   0.00000000e+00  0.00000000e+00  2.00000000e+01 -5.33333333e+01]
 [-7.69230769e+01  6.59340659e+01  0.00000000e+00  0.00000000e+00
   2.96703297e+02 -1.42857143e+02 -2.19780220e+02  7.69230769e+01]
 [ 7.69230769e+01 -2.19780220e+02  0.00000000e+00  0.00000000e+00
  -1.42857143e+02  2.96703297e+02  6.59340659e+01 -7.69230769e+01]
 [ 0.00000000e+00 -8.59340659e+01 -2.00000000e+01  2.00000000e+01
  -2.19780220e+02  6.59340659e+01  2.39780220e+02 -2.22044605e-15]
 [-9.02564103e+01  0.00000000e+00  1.33333333e+01 -5.33333333e+01
   

In [3]:
print(fglobal)
print(desl)
print(tensao)

[  0.           0.         166.66666667   0.         166.66666667
 167.66666667 333.33333333 334.33333333]
[ 0.          0.          5.65994167  0.         10.3386225   2.48873583
 10.65419167 -2.64839167]
[[   2.          383.27583333  218.10916667  299.9425    ]
 [   1.          533.10333333 -131.56333333  199.77      ]
 [   1.          233.44833333  567.78166667  400.115     ]
 [   2.          383.27583333  218.10916667  299.9425    ]]
